# Pengujian Model Apple Quality dengan TensorFlow Serving dan Web Interface

Notebook ini digunakan untuk menguji model klasifikasi kualitas apel yang telah di-deploy menggunakan TensorFlow Serving dan FastAPI pada Railway. Model ini dapat memprediksi apakah kualitas apel "good" atau "bad" berdasarkan fitur-fitur seperti ukuran, berat, tingkat kemanisan, kerenyahan, kejusan, kematangan, dan keasaman.

## Alur Pengujian

1. **Pengujian Web Interface**: 
   - Mengakses aplikasi web pada domain Railway
   - Menggunakan form untuk mengirim input karakteristik apel
   - Melihat hasil prediksi secara visual

2. **Pengujian API Endpoint**:
   - Melakukan preprocessing data
   - Mengirim permintaan ke endpoint API
   - Menganalisis respons JSON

3. **Pengujian TensorFlow Serving Langsung**:
   - Mengirim data ke TensorFlow Serving melalui REST API
   - Menerima hasil prediksi dalam format JSON
   - Menginterpretasikan hasil prediksi

## Cara Akses Web Interface

Aplikasi web dapat diakses melalui URL berikut:
```
https://apple-quality-model-kstarid.up.railway.app/
```

## Catatan Penting

- Pastikan model sudah ter-deploy dengan benar ke Railway
- Struktur direktori model harus sesuai dengan yang diharapkan oleh TensorFlow Serving
- Preprocessing data harus konsisten dengan yang digunakan saat training


In [ ]:
# Fungsi untuk melakukan prediksi menggunakan API endpoint
def predict_quality_api(data):
    print("=== PENGUJIAN DENGAN FASTAPI ENDPOINT ===")
    
    # Step 1: Persiapkan data
    print("\n1. Data yang Dikirim:")
    print(json.dumps(data, indent=2))
    
    # Step 2: Kirim request ke API
    print("\n2. Mengirim Request ke API:")
    print(f"URL: {API_URL}")
    response = requests.post(API_URL, json=data)
    print(f"Status Code: {response.status_code}")
    
    # Step 3: Parse response
    print("\n3. Response dari API:")
    print(f"Response Raw: {response.text}")
    
    try:
        result = response.json()
        print(f"Response JSON: {json.dumps(result, indent=2)}")
        
        # Step 4: Interpretasi hasil
        print("\n4. Interpretasi Hasil:")
        print(f"Label: {result['prediction']} (Probability: {result['probability']:.4f})")
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None


In [ ]:
# Contoh data apel berkualitas baik
good_apple = {
    "A_id": 1,
    "Size": 0.8,
    "Weight": 0.7,
    "Sweetness": 0.9, 
    "Crunchiness": 0.8,
    "Juiciness": 0.9,
    "Ripeness": 0.7,
    "Acidity": 0.5
}

# Contoh data apel berkualitas buruk
bad_apple = {
    "A_id": 2,
    "Size": 0.3,
    "Weight": 0.2,
    "Sweetness": 0.2, 
    "Crunchiness": 0.3,
    "Juiciness": 0.1,
    "Ripeness": 0.4,
    "Acidity": 0.8
}

# Uji dengan data apel berkualitas baik
print("\n==== TEST APEL BERKUALITAS BAIK ====\n")
good_result = predict_quality_api(good_apple)


In [ ]:
# Uji dengan data apel berkualitas buruk
print("\n==== TEST APEL BERKUALITAS BURUK ====\n")
bad_result = predict_quality_api(bad_apple)


In [ ]:
# Fungsi untuk melakukan preprocessing data
def preprocess_data(data):
    # Contoh data yang sudah dinormalisasi (simulasi)
    processed_data = {}
    
    # Proses fitur numerik
    for feature in NUMERIC_FEATURES:
        processed_data[transformed_name(feature)] = [float(data[feature])]
    
    # Proses Acidity
    processed_data[transformed_name('Acidity')] = [float(data['Acidity'])]
    
    return processed_data

# Fungsi untuk melakukan prediksi langsung ke TensorFlow Serving
def predict_quality_tf_serving(data):
    print("=== PENGUJIAN LANGSUNG KE TENSORFLOW SERVING ===")
    
    # Step 1: Preprocessing data
    print("\n1. Preprocessing Data:")
    print(f"Data mentah: {data}")
    processed_data = preprocess_data(data)
    print(f"Data setelah preprocessing: {processed_data}")
    
    # Step 2: Format data untuk TensorFlow Serving
    print("\n2. Memformat Data untuk TensorFlow Serving:")
    instances = {
        'instances': [processed_data]
    }
    print(f"Request JSON: {json.dumps(instances, indent=2)}")
    
    # Step 3: Kirim request ke server
    print("\n3. Mengirim Request ke TensorFlow Serving:")
    print(f"URL: {TF_SERVING_URL}")
    response = requests.post(TF_SERVING_URL, json=instances)
    print(f"Status Code: {response.status_code}")
    
    # Step 4: Parse response
    print("\n4. Menerima dan Parsing Response:")
    print(f"Response Raw: {response.text}")
    
    try:
        result = json.loads(response.text)
        print(f"Response JSON: {json.dumps(result, indent=2)}")
        prediction = result['predictions'][0][0]
        print(f"Nilai Prediksi: {prediction}")
        
        # Step 5: Konversi probability ke label
        print("\n5. Interpretasi Hasil:")
        label = "good" if prediction > 0.5 else "bad"
        probability = prediction if prediction > 0.5 else 1 - prediction
        print(f"Label: {label} (Probability: {probability:.4f})")
        
        return {
            "label": label,
            "probability": probability,
            "raw_prediction": prediction
        }
    except Exception as e:
        print(f"Error: {e}")
        print("Note: Make sure TensorFlow Serving is running with the correct model.")
        return None


In [ ]:
# Uji dengan data apel berkualitas baik menggunakan TensorFlow Serving langsung
print("\n==== TEST APEL BERKUALITAS BAIK (TF SERVING) ====\n")
tf_good_result = predict_quality_tf_serving(good_apple)


In [ ]:
# Uji dengan data apel berkualitas buruk menggunakan TensorFlow Serving langsung
print("\n==== TEST APEL BERKUALITAS BURUK (TF SERVING) ====\n")
tf_bad_result = predict_quality_tf_serving(bad_apple)


In [ ]:
# Ringkasan hasil
print("=== RINGKASAN HASIL PENGUJIAN ===")

print("\nHasil FastAPI Endpoint:")
print(f"- Apel Berkualitas Baik: {good_result['prediction'] if good_result else 'Error'} (Probability: {good_result['probability']:.4f} if good_result else 'N/A')")
print(f"- Apel Berkualitas Buruk: {bad_result['prediction'] if bad_result else 'Error'} (Probability: {bad_result['probability']:.4f} if bad_result else 'N/A')")

print("\nHasil TensorFlow Serving:")
print(f"- Apel Berkualitas Baik: {tf_good_result['label'] if tf_good_result else 'Error'} (Probability: {tf_good_result['probability']:.4f} if tf_good_result else 'N/A')")
print(f"- Apel Berkualitas Buruk: {tf_bad_result['label'] if tf_bad_result else 'Error'} (Probability: {tf_bad_result['probability']:.4f} if tf_bad_result else 'N/A')")

print("\n=== KESIMPULAN ===")
print("Kedua pendekatan (FastAPI dan TensorFlow Serving langsung) menunjukkan hasil yang konsisten")
print("Model berhasil di-deploy dan dapat diakses melalui web interface maupun API endpoint")


In [ ]:
import json
import requests
import numpy as np
import pandas as pd
from apple_transform import NUMERIC_FEATURES, transformed_name

# URL untuk Web App API (FastAPI endpoint)
API_URL = 'https://apple-quality-model-kstarid.up.railway.app/predict/'

# URL untuk TensorFlow Serving langsung
TF_SERVING_URL = 'https://apple-quality-model-kstarid.up.railway.app/v1/models/apple_quality_model:predict'


=== MULAI PROSES TESTING ===
Menjalankan prediksi untuk sampel apel...
=== PROSES PREDIKSI KUALITAS APEL ===

1. Preprocessing Data:
Data mentah: {'A_id': 1, 'Size': 0.6, 'Weight': 0.6, 'Sweetness': 0.7, 'Crunchiness': 0.8, 'Juiciness': 0.9, 'Ripeness': 0.7, 'Acidity': 0.5}
Data setelah preprocessing: {'A_id_xf': [1.0], 'Size_xf': [0.6], 'Weight_xf': [0.6], 'Sweetness_xf': [0.7], 'Crunchiness_xf': [0.8], 'Juiciness_xf': [0.9], 'Ripeness_xf': [0.7], 'Acidity_xf': [0.5]}

2. Memformat Data untuk TensorFlow Serving:
Request JSON: {
  "instances": [
    {
      "A_id_xf": [
        1.0
      ],
      "Size_xf": [
        0.6
      ],
      "Weight_xf": [
        0.6
      ],
      "Sweetness_xf": [
        0.7
      ],
      "Crunchiness_xf": [
        0.8
      ],
      "Juiciness_xf": [
        0.9
      ],
      "Ripeness_xf": [
        0.7
      ],
      "Acidity_xf": [
        0.5
      ]
    }
  ]
}

3. Mengirim Request ke TensorFlow Serving:
URL: https://apple-model.up.railway.app/